In [ ]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from functions_utils import summarize_text_bart

C:\Users\48798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df[['text', 'summary']]

In [3]:
df = df[df['summary'].apply(len) <= 200]

In [4]:
import nltk


In [5]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments


In [6]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)

# Tokenize dataset
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def summarize_text(text, trained_model):
    """Generate a summary for the given text."""
    inputs = tokenizer(text, max_length=512, return_tensors="pt", truncation=True).to(device)
    summary_ids = trained_model.generate(inputs["input_ids"], max_length=128, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


tokenizer = BartTokenizer.from_pretrained("./fine_tuned_bart")
model = BartForConditionalGeneration.from_pretrained("./fine_tuned_bart").to(device)

C:\Users\48798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [8]:
sample = df.iloc[26, 0]
generated_summary = summarize_text(sample, model)
print(sample)
print("Generated Summary:", generated_summary)

C:\Users\48798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1527: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


South Korea has accused North Korea of planting landmines that maimed two soldiers on border patrol, sending military tensions on the Korean peninsula soaring as it threatened to make Pyongyang pay a "harsh price".

The Defence Ministry said it believed three landmines exploded in the incident last Tuesday, hitting a patrol in the demilitarised zone (DMZ) - a buffer zone stretching two kilometres on either side of the actual frontier line dividing the two Koreas.

"We are certain they were North Korean landmines planted with an intention to kill by our enemies who sneaked across the military border," ministry spokesman Kim Min-Seok told reporters.

One soldier underwent a double leg amputation, while the other had one leg removed.

In a statement, the South's Joint Chiefs of Staff said its military would make North Korea "pay a harsh price proportionate for the provocation it made".

Describing the attack as a "baseless act" and "wanton violation" of non-aggression accords, the stateme

In [9]:
df.to_csv('data_eva.csv', index = False)

In [10]:
from datetime import datetime


df.reset_index(drop = True, inplace = True)
df['gen_sum_bart'] = ''

start_time = datetime.now()
for i in range(0, 10):
    sample = df.iloc[i, 0]
    generated_summary = summarize_text(sample, model)

df.to_csv('dane_bert_comp.csv')

end_time = datetime.now()
execution_time = (end_time - start_time).total_seconds()
print(f"Czas predykcji (10 przykładów): {execution_time:.6f} sekund")

Czas predykcji (10 przykładów): 136.095759 sekund


In [11]:
summarize_text(str(df.sample(1).iloc[0]),model)

''